In [2]:
import sys
import regex
import cgatcore.iotools as iotools
import pysam
import logging
import argparse

In [2]:
tab = str.maketrans("ACTG", "TGAC")

def reverse_complement_table(seq):
    return seq.translate(tab)[::-1]


outfile = iotools.open_file("polya_revcomp.fastq", "w")
log =  iotools.open_file("revcomp" + ".log","w")
n = 0
y = 0
with pysam.FastxFile("input.fastq") as fh:
    
    for record in fh:
        y +=1
        if len(record.sequence) < 300:
            pass
        else:
            seq = record.sequence[30:300]
            m=regex.findall("(TTTTTTTTTTTTTTTTTTTT){e<=0}", str(seq))
            if m:
                n +=1
                sequence = reverse_complement_table(str(record.sequence))
                outfile.write("@%s\n%s\n+\n%s\n" % (record.name, sequence, record.quality))
            else:
                seq = record.sequence[-30:-200:]
                m=regex.findall("(AAAAAAAAAAAAAAAAAAAA){e<=0}", str(seq))
                if m:
                    n +=1
                    outfile.write("@%s\n%s\n+\n%s\n" % (record.name, record.sequence, record.quality))
                            
print(n)
print(y)
outfile.close()
log.close()

1262869
6262353


In [3]:
tab = str.maketrans("ACTG", "TGAC")

def reverse_complement_table(seq):
    return seq.translate(tab)[::-1]

def most_common(lst):
    return max(set(lst), key=lst.count)

outfile = open("polya_umi.fastq", "w")

n = 0

collapsed_trimers = []
umis = []

with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:
        n += 1
        seq_nano = record.sequence
        
        m=regex.finditer("(GTACTCTGCGTTGATACCACTGCTT){e<=0}", str(record.sequence))
        
        for i in m:
            umi = seq_nano[i.start()-30:i.start()]
            #print(umi)
            single_nuc = []
            trimers = [umi[i:i+3] for i in range(0, len(umi), 3)]
            
            for trimer in trimers:
                single_nuc.append(trimer)
            
            collapse = "".join(single_nuc)
            collapsed_trimers.append(collapse)
            umis.append(umi)
            outfile.write("@%s\n%s\n+\n%s\n" % (record.name, record.sequence, record.quality))

            
            
            
#print(collapsed_trimers)



In [4]:
# count the number of perfect 
from collections import Counter
counts = []
n = 0
for sequence in collapsed_trimers:
    n += 1
    count = sum(1 for a, b in zip("GGGAAACCCTTTGGGCCCTTTAAACCCTTT", sequence) if a != b)
    #print(count)
    counts.append(count)
print(Counter(counts))

print("The percent of perfect UMIs with trimers: %s"%(Counter(counts)[0]/n*100))
print("The percent of one missmatch UMI with trimers: %s"%(Counter(counts)[1]/n*100))
print("The percent of two missmatch UMI with trimers: %s"%(Counter(counts)[2]/n*100))
print("The percent of three missmatch UMI with trimers: %s"%(Counter(counts)[3]/n*100))
print("The percent of 4 missmatch UMI with trimers: %s"%(Counter(counts)[4]/n*100))
print("The percent of 5 missmatch UMI with trimers: %s"%(Counter(counts)[5]/n*100))
print("The percent of 6 missmatch UMI with trimers: %s"%(Counter(counts)[6]/n*100))
print("The percent of 7 missmatch UMI with trimers: %s"%(Counter(counts)[7]/n*100))
print("The percent of 8 missmatch UMI with trimers: %s"%(Counter(counts)[8]/n*100))
print("The percent of 9 missmatch UMI with trimers: %s"%(Counter(counts)[9]/n*100))
print("The percent of 10 missmatch UMI with trimers: %s"%(Counter(counts)[10]/n*100))


Counter({0: 305873, 4: 94483, 6: 68883, 3: 64168, 7: 38812, 5: 30359, 2: 26473, 1: 26338, 9: 26021, 8: 23806, 10: 19370, 11: 14578, 12: 10537, 14: 10476, 13: 8355, 15: 7526, 18: 5294, 16: 5127, 17: 4781, 19: 3432, 21: 2757, 20: 2690, 22: 2018, 23: 1610, 24: 1263, 25: 1014, 26: 937, 27: 392, 29: 248, 28: 241, 30: 110})
The percent of perfect UMIs with trimers: 37.85688117905076
The percent of one missmatch UMI with trimers: 3.2597664275494695
The percent of two missmatch UMI with trimers: 3.2764749273489677
The percent of three missmatch UMI with trimers: 7.941859371364354
The percent of 4 missmatch UMI with trimers: 11.693845826340517
The percent of 5 missmatch UMI with trimers: 3.7574321882441466
The percent of 6 missmatch UMI with trimers: 8.52541919769497
The percent of 7 missmatch UMI with trimers: 4.803631809023085
The percent of 8 missmatch UMI with trimers: 2.946389231309996
The percent of 9 missmatch UMI with trimers: 3.2205323946869444
The percent of 10 missmatch UMI with trim

In [9]:
# Calculate the recovery of each umi that isnt perfect

umi_insertions = []
for collapse, umi in zip(collapsed_trimers, umis):
    count = sum(1 for a, b in zip("GACTGCTACT", umi) if a != b)
    
    if count > 0:
        umi_insertions.append(umi)
#umi_insertions

In [10]:
import collections

def allCharactersSame(s) :
    n = len(s)
    for i in range(1, n) :
        if s[i] != s[0] :
            return False
 
    return True

def remove_point_mutations(umi):
    
    point_remove = []
    
    trimers = [umi[i:i+3] for i in range(0, len(umi), 3)]

    n = 0
    error_counter = 0
    for trimer in trimers:

        if allCharactersSame(trimer):
            point_remove.append(trimer)
        if not allCharactersSame(trimer):
            base = collections.Counter(trimer).most_common(1)[0][0]
            base = base+base+base
            point_remove.append(base)
            n += 1
    return("".join(point_remove), n)


def remove_indels(x, umi, first):
    
    substr = umi[x:x+3]
    
    if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr) and first:
        pass
    
#    else:
#        # correct for point mutations at beginning
#        base = collections.Counter(substr).most_common(1)[0][0]
#        substr = base+base+base
#        first = False
        
    
    if not ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
        # if there is an error then check to see if the next base is a perfect match
        substr = umi[x+1:x+4]
        
        if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
            # then check to see if perfect trimer
            pass

        elif not ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
            # if not then check to see if there is a 
            substr = umi[x+2:x+5]

            
            
    return(substr)




def correct_umi(umis):
    n = 0
    corrected_umis = []
    for umi in umis:
        n +=1

        new_umi = []
        umi, errors = remove_point_mutations(umi)
        if errors > 2:
            pass
        else:
            for x in range(0, len(umi)):
                if x % 3 == 0:

                    if x == 0:
                        #print(x, umi)
                        sub_umi = remove_indels(x, umi, first=True)
                    else:
                        sub_umi = remove_indels(x, umi, first=False)

                    new_umi.append(sub_umi)

            new_umi = "".join(new_umi)
            final_umi = new_umi[:30]
            corrected_umis.append(final_umi[::3])
    return(corrected_umis)


corrected_umis = correct_umi(umi_insertions)

print(umi_insertions[:10])
print(corrected_umis[:10])


umis_not_correct = []

perfect = 0
total = 0
for umi in corrected_umis:
    total += 1
    if umi == 'GACTGCTACT':
        perfect += 1
    else:
        umis_not_correct.append(umi)

print(perfect)
print(total)
print("Total UMIs that are free of errors: {}".format(perfect/total *100))
print("Total UMIs that have errors: {}".format((total - perfect)/total *100))

['TGGGAAACCCTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTGGGCCCTTTAAAACCCTTT', 'GGAAACCCTTTGGGCCCTTTAAACCCTTTT', 'AAACCCTCTTTGGGCCCTTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'TTGGGAACCTTTGGGCTCTTTAAACCCTTT']
['GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT']
354115
380687
Total UMIs that are free of errors: 93.0199875488262
Total UMIs that have errors: 6.980012451173799


In [7]:
# how many of the umis that are not correct have a lV distance
l_dist = []
import Levenshtein

for umi in umis_not_correct:
    ld = Levenshtein.distance(umi, 'GACTGCTACT')
    l_dist.append(ld)

umi_counter = Counter(l_dist)
umi_counter

Counter({2: 83880,
         1: 54386,
         3: 20831,
         4: 11518,
         8: 345,
         5: 5311,
         7: 1205,
         6: 2306,
         9: 17})

In [9]:
# How many idels vs point mutations

umi_insertions = []
def allCharactersSame(s) :
    n = len(s)
    for i in range(1, n) :
        if s[i] != s[0] :
            return False
 
    return True


indel = 0
point = 0
total = 0
lines = 0
with pysam.FastxFile("polya_umi.fastq") as fh:
    
    for record in fh:
        lines += 1
        seq_nano = record.sequence
        
        m=regex.finditer("(GTACTCTGCGTTGATACCACTGCTT){e<=0}", str(record.sequence))
        
        first = 0
        for i in m:
            first += 1
            if first == 1:
                total +=1
            umi = seq_nano[i.start()-30:i.start()]
            #print(umi)
            single_nuc = []

            trimers = [umi[i:i+3] for i in range(0, len(umi), 3)]

            n = 0
            error_counter = 0
            for trimer in trimers:

                if allCharactersSame(trimer):
                    pass
                if not allCharactersSame(trimer):
                    #print(trimer)
                    n += 1
                    #print(n)
                    if n > 1:
                        error_counter += 1
            
            if error_counter ==1:
                point += 1
            if error_counter ==2:
                point += 1
            if error_counter ==3:
                point += 1
        
            if error_counter > 3:
                indel += 1
                umi_insertions.append(umi)
print(lines)
print(total)
print(point)
print(indel)

print("Total UMIs that are free of errors: {}".format((lines-(point+indel))/lines *100))
print("Total UMIs that have either point or indels: {}".format((point+indel)/lines *100))
print("Total UMIs that have a point error: {}".format(point/lines *100))
print("Total UMIs that have either indels error: {}".format(indel/lines *100))
                        
        

316957
316957
74237
91270
Total UMIs that are free of errors: 47.782506775367004
Total UMIs that have either point or indels: 52.217493224632996
Total UMIs that have a point error: 23.421789075489734
Total UMIs that have either indels error: 28.79570414914326


In [36]:
# Determine how many of the indel sequences are corrected by using homotrimer 
import collections

def allCharactersSame(s) :
    n = len(s)
    for i in range(1, n) :
        if s[i] != s[0] :
            return False
 
    return True

def remove_point_mutations(umi):
    
    point_remove = []
    
    trimers = [umi[i:i+3] for i in range(0, len(umi), 3)]

    n = 0
    error_counter = 0
    for trimer in trimers:

        if allCharactersSame(trimer):
            point_remove.append(trimer)
        if not allCharactersSame(trimer):
            base = collections.Counter(trimer).most_common(1)[0][0]
            base = base+base+base
            point_remove.append(base)
            n += 1
            #print(n)
            if n > 1:
                error_counter += 1
    return("".join(point_remove))


def remove_indels(x, umi, first):
    
    substr = umi[x:x+3]
    
    if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr) and first:
        pass
    
#    else:
#        # correct for point mutations at beginning
#        base = collections.Counter(substr).most_common(1)[0][0]
#        substr = base+base+base
#        first = False
        
    
    if not ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
        # if there is an error then check to see if the next base is a perfect match
        substr = umi[x+1:x+4]
        
        if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
            # then check to see if perfect trimer
            pass

        elif not ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
            # if not then check to see if there is a 
            substr = umi[x+2:x+5]

            
            
    return(substr)




def correct_umi(umis):
    n = 0
    corrected_umis = []
    for umi in umis:
        n +=1

        new_umi = []
        umi = remove_point_mutations(umi)
        for x in range(0, len(umi)):
            if x % 3 == 0:

                if x == 0:
                    #print(x, umi)
                    sub_umi = remove_indels(x, umi, first=True)
                else:
                    sub_umi = remove_indels(x, umi, first=False)

                new_umi.append(sub_umi)

        new_umi = "".join(new_umi)
        final_umi = new_umi[:30]
        corrected_umis.append(final_umi[::3])
    return(corrected_umis)


corrected_umis = correct_umi(umi_insertions)

print(umi_insertions[:10])
print(corrected_umis[:10])


umis_not_correct = []
umis_correct = []

perfect = 0
total = 0
for umi in corrected_umis:
    total += 1
    if umi == 'GACTGCTACT':
        perfect += 1
        umis_correct.append(umi)
    else:
        umis_not_correct.append(umi)
        
print("Total UMIs that are free of errors: {}".format(perfect/total *100))
print("Total UMIs that have errors: {}".format((total - perfect)/total *100))

['TGGGAAACCCTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTGGGCCCTTTAAAACCCTTT', 'GGAAACCCTTTGGGCCCTTTAAACCCTTTT', 'AAACCCTCTTTGGGCCCTTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'GGGAAACCCTTTGGGCCCTTTAAACCCTTT', 'TTGGGAACCTTTGGGCTCTTTAAACCCTTT']
['GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'ACTTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'GACTGCTACT', 'TGCTGCTACT']
Total UMIs that are free of errors: 77.74406773367002
Total UMIs that have errors: 22.255932266329978


In [31]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]


from collections import Counter
counts = []
n = 0

for umi in umis_not_correct:
    
    
    n +=1
    counts.append(levenshteinDistance("GACTGCTACT", umi))

umi_counter = Counter(counts)

umi_counter

Counter({1: 15695,
         2: 6550,
         5: 1406,
         4: 1347,
         6: 264,
         3: 1154,
         7: 133,
         8: 22,
         9: 1})

In [32]:
one_two = umi_counter[1] + umi_counter[2]
umis_total = sum(umi_counter.values())

print("Percent of UMIs that have an indel but are only one or two away from the perfect so should be corrected with UMI-tools or mcl-UMI: {}".format(one_two/umis_total*100))


Percent of UMIs that have an indel but are only one or two away from the perfect so should be corrected with UMI-tools or mcl-UMI: 83.71594159265392


# Older code below

In [415]:
# Basecalling accuracy


def basecall_accuracy(barcodes):
    errors_per_bc = []
    error_bc = 0
    n = 0
    error = 0
    perfect = 0
    i = 0
    barcodes = "".join(barcodes)


    for x in range(0,len(barcodes)):

        substr = barcodes[x:x+3]

        if i % 3 == 1:
            pass
        elif i % 3 == 2:
            pass
        else:
            n += 1
            if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
                perfect +=1
            else:
                error +=1
                error_bc +=1
        i +=1
    errors_per_bc.append(error_bc)
    error_bc = 0
            
    accuracy = (error/n) * 100
    print(perfect)
    print(error)
    print(accuracy)
    


In [416]:
# umi_length = 12
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 36]
            
            if umi != "":
                if len(umi) != 36:
                    pass
                else:
                    barcodes.append(umi)
#print(barcodes)
basecall_accuracy(barcodes)

1980965
2565799
56.431321264969995


In [417]:
# umi_length = 11
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 33]
            
            if umi != "":
                if len(umi) != 33:
                    pass
                else:
                    barcodes.append(umi)
#print(barcodes)
basecall_accuracy(barcodes)

1881592
2287793
54.871234006933875


In [418]:
# umi_length = 10
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 30]
            
            if umi != "":
                if len(umi) != 30:
                    pass
                else:
                    barcodes.append(umi)
#print(barcodes)
basecall_accuracy(barcodes)

1756756
2035934
53.680474808117715


In [419]:
# umi_length = 9
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 27]
            
            if umi != "":
                if len(umi) != 27:
                    pass
                else:
                    barcodes.append(umi)
#print(barcodes)
basecall_accuracy(barcodes)

1617590
1797001
52.627122838430715


In [420]:
# umi_length = 8
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 24]
            
            if umi != "":
                if len(umi) != 24:
                    pass
                else:
                    barcodes.append(umi)
#print(barcodes)
basecall_accuracy(barcodes)

1484211
1551269
51.10456995269283


In [421]:
# umi_length = 7
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 21]
            
            if umi != "":
                if len(umi) != 21:
                    pass
                else:
                    barcodes.append(umi)
#print(barcodes)
basecall_accuracy(barcodes)

1358513
1297672
48.85472962161897


In [422]:
# umi_length = 6
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 18]
            
            if umi != "":
                if len(umi) != 18:
                    pass
                else:
                    barcodes.append(umi)
#print(barcodes)
basecall_accuracy(barcodes)

1214271
1062837
46.67486127140214


In [423]:
# umi_length = 5
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 15]
            
            if umi != "":
                if len(umi) != 15:
                    pass
                else:
                    barcodes.append(umi)
#print(barcodes)
basecall_accuracy(barcodes)

1068565
829450
43.70091911813131


In [385]:
# What is the indel error rate at the base level?
# Reported to be 2.5% in https://www.nature.com/articles/s41467-020-20340-8
# to calculate this if we see more than one trimer having errors in a row then we state this as an indel
# iteraste through count if error occurs in more than one trimer, then when it doesnt reset, count number of indels
# as a percentage of bases. 

def indel_frequency(barcodes, umi_length=10):
    errors_per_bc = []
    error_bc = 0
    n = 0
    error = 0
    perfect = 0
    
    record_position_error = 0
    
    first = True
    
    point = 0
    indel = 0
    
    i = 0
    barcodes = "".join(barcodes)


    for x in range(0,len(barcodes)):

        substr = barcodes[x:x+3]
        
        if i % 3 == 1:
            pass
        elif i % 3 == 2:
            pass
        else:
            n += 1
            if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
                perfect +=1
                first = True
            else:
                
                if first:
                    point += 1
                    first = False
                else:
                    indel += 1
                    point -= 1
                    break
                error +=1
                error_bc +=1
        i +=1
    errors_per_bc.append(error_bc)
    error_bc = 0
            
    accuracy = (error/n) * 100
    #print(perfect)
    #print(error)
    #print("point: {}".format(point))
    #print("indel: {}".format(indel))
    #print(accuracy)
    return(umi_length, indel, point)


# indel - more than or equal to two errors but errors next to eachother
indel_frequency("AAATTTCCCTTTGGGATTTGGGAAATTTCC")

indel_frequency("AAATTTCCCTTTGGATTTGGGAAATTTCCC")


SyntaxError: invalid syntax (1656162878.py, line 39)

In [401]:
# perfect
#indel_frequency("AAATTTCCCTTTGGGTTTGGGAAATTTCCC")
# point - only one error

total_trimers = 0
total_indel = 0
total_point = 0
# umi_length = 10
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 30]
            
            if umi != "":
                if len(umi) != 30:
                    pass
                else:
                    barcodes.append(umi)
#barcodes




trimers: 29990
indel: 2999
point: 0
point frequency: 0.0
indel frequency: 10.0


In [408]:
n = 0
for barcode in barcodes:
    n +=1
    if n < 30:
        umi_length, indel, point = indel_frequency(barcodes, umi_length=10)
        print(barcode)
        total_trimers += umi_length
        total_indel += indel
        total_point += point
    else:
        pass
print("trimers: {}".format(total_trimers))
print("indel: {}".format(total_indel))
print("point: {}".format(total_point))
print("point frequency: {}".format((total_point/total_trimers)*100))
print("indel frequency: {}".format((total_indel/total_trimers)*100))

CCCTTTTGGGAAACCCTTTGGGCCCTTTAA
TCTTTGGGAAACCCTTTGGCCCTTTAAACC
CCCTTTGGGAAAACCCTTGGGCCCTTTAAA
CTCCTTTGGGGGAACCCTTTGGGCTCCTTT
CCTCTTTGGGAAACCCTTTGGGCCCTTTAA
CCCTTTGGGAAACCCTTTTGGGCCCTTTTA
CCCTTTGGGAAACCCTTGGGCCCTTTTAAA
CCCTTTAGTTCCCTTGGGCCCTTTTTAAAC
CCCTTTGGGAAACCCTTGGGCCTTTAAACC
CCTTTGGGAAACCCTTTGGGCCCTTTAAAC
CCCTTTGGGAAACCCTTTGGGCCCTTTAAA
CCCTTTGGAAACCCTTGGGCTCCTAAAAAC
CCCTTTTGCGAAACCTTTGGGCCCTTTAAA
CCCTTTGGGAAACCCTTTGGGCCCTTAAAA
CCCTTTTGTAAAACCCTTTGGGCTCCTTTA
CCCTTTGGGAAACCCTTTTGGGCTCCTTTA
CCCTTTTTGGGAAACCCTAAAGCCCTTTAG
CCCTTTGGGAAACCCTTTGGGCCCTTTAAA
CCCTTTGGGAAACCCTTTGGGCCCTTTAAA
CCCTTGGGAAACCCTTTTGGGTCTCTTTAA
CCCTTTGGGAAACCCTTGGGCCCTTTAAAC
CCCTTTGGGAAACCCTTTGGGCCCTTTAAA
CCCTTGGGAAACCCTTTGGGCCCTCTTAAA
CCCTTTGGGAAACCCCTTGGGCCCTTTAAA
CTCCTTGGGAAACCCTTTGGGCCCTTTAAA
CCCTTTGGGGAAACCCTTGGGCCCTTGAAC
CCCTTTGGGAAACCCTTGGGCCCTTCTTTA
CCCTTTGTGGGAAAACCCTTTGGGCCCTTT
CCTTTGGGAAACCCTTTGGGCCCTTTAAAC
trimers: 580
indel: 58
point: 0
point frequency: 0.0
indel frequency: 10.0


In [ ]:
indel_frequency

(10, 0, 1)

In [409]:
# perfect
#indel_frequency("AAATTTCCCTTTGGGTTTGGGAAATTTCCC")
# point - only one error

total_trimers = 0
total_indel = 0
total_point = 0
# umi_length = 10
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 36]
            
            if umi != "":
                if len(umi) != 36:
                    pass
                else:
                    barcodes.append(umi)
#barcodes

n = 0
for barcode in barcodes:
    n += 1
    if n < 3000:
        umi_length, indel, point = indel_frequency(barcodes, umi_length=12)
        total_trimers += umi_length
        total_indel += indel
        total_point += point
    
print("trimers: {}".format(total_trimers))
print("indel: {}".format(total_indel))
print("point: {}".format(total_point))
print("point frequency: {}".format((total_point/total_trimers)*100))
print("indel frequency: {}".format((total_indel/total_trimers)*100))

trimers: 35988
indel: 2999
point: 0
point frequency: 0.0
indel frequency: 8.333333333333332


In [410]:

total_trimers = 0
total_indel = 0
total_point = 0
# umi_length = 10
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 33]
            
            if umi != "":
                if len(umi) != 33:
                    pass
                else:
                    barcodes.append(umi)
#barcodes

n = 0
for barcode in barcodes:
    n +=1
    if n < 3000:
        umi_length, indel, point = indel_frequency(barcodes, umi_length=11)
        total_trimers += umi_length
        total_indel += indel
        total_point += point
    
print("trimers: {}".format(total_trimers))
print("indel: {}".format(total_indel))
print("point: {}".format(total_point))
print("point frequency: {}".format((total_point/total_trimers)*100))
print("indel frequency: {}".format((total_indel/total_trimers)*100))

trimers: 32989
indel: 2999
point: 0
point frequency: 0.0
indel frequency: 9.090909090909092


In [424]:

total_trimers = 0
total_indel = 0
total_point = 0
# umi_length = 10
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 27]
            
            if umi != "":
                if len(umi) != 27:
                    pass
                else:
                    barcodes.append(umi)
#barcodes

n = 0
for barcode in barcodes:
    n +=1
    if n < 3000:
        umi_length, indel, point = indel_frequency(barcodes, umi_length=9)
        total_trimers += umi_length
        total_indel += indel
        total_point += point
    
print("trimers: {}".format(total_trimers))
print("indel: {}".format(total_indel))
print("point: {}".format(total_point))
print("point frequency: {}".format((total_point/total_trimers)*100))
print("indel frequency: {}".format((total_indel/total_trimers)*100))

trimers: 26991
indel: 2999
point: 0
point frequency: 0.0
indel frequency: 11.11111111111111


In [469]:
barcodes = []
with pysam.FastxFile("polya_revcomp.fastq") as fh:
    
    for record in fh:

        seq_nano = record.sequence
        
        m=regex.finditer("(AAAAAAAAAAAAAAAG){e<=0}", str(record.sequence))
        
        for i in m:
            
            umi = seq_nano[i.end():i.end() + 36]
            
            if umi != "":
                if len(umi) != 36:
                    pass
                else:
                    barcodes.append(umi)
#barcodes

In [511]:
# Randomly select every 3rd base and determine the ld distance

def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]


from collections import Counter
counts = []
n = 0
for umi in barcodes:
    
    
    n +=1
    counts.append(levenshteinDistance("CTGACTGCTA", umi[::3]))

Counter(counts)

Counter({2: 185222,
         3: 76879,
         4: 62493,
         6: 12435,
         10: 905,
         5: 29068,
         8: 3992,
         9: 1541,
         7: 6362})

In [522]:
Counter(counts)[10]/n*100

0.23885119174868102

In [523]:
# Identify insertions and remove them

import collections

def allCharactersSame(s) :
    n = len(s)
    for i in range(1, n) :
        if s[i] != s[0] :
            return False
 
    return True

def remove_point_mutations(umi):
    
    point_remove = []
    
    trimers = [umi[i:i+3] for i in range(0, len(umi), 3)]

    n = 0
    error_counter = 0
    for trimer in trimers:

        if allCharactersSame(trimer):
            point_remove.append(trimer)
        if not allCharactersSame(trimer):
            base = collections.Counter(trimer).most_common(1)[0][0]
            base = base+base+base
            point_remove.append(base)
            n += 1
            #print(n)
            if n > 1:
                error_counter += 1
    return("".join(point_remove))


def remove_indels(x, umi, first):
    
    substr = umi[x:x+3]
    
    if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr) and first:
        pass
    
    else:
        # correct for point mutations at beginning
        base = collections.Counter(substr).most_common(1)[0][0]
        substr = base+base+base
        first = False
        
    
    if not ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
        # if there is an error then check to see if the next base is a perfect match
        substr = umi[x+1:x+4]
        
        if ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
            # then check to see if perfect trimer
            pass

        elif not ("CCC" in substr or "GGG" in substr or "AAA" in substr or "TTT" in substr):
            # if not then check to see if there is a 
            substr = umi[x+2:x+5]

            
            
    return(substr, indel_1, indel_2)




def correct_umi(umis):
    n = 0
    corrected_umis = []
    for umi in umis:
        n +=1

        new_umi = []
        #umi = remove_point_mutations(umi)
        for x in range(0, len(umi)):
            if x % 3 == 0:


                if x == 0:
                    sub_umi, indel_1, indel_2 = remove_indels(x, umi, first=True)
                else:
                    sub_umi, indel_1, indel_2 = remove_indels(x, umi, first=False)

                new_umi.append(sub_umi)

        new_umi = "".join(new_umi)
        final_umi = new_umi[:30]
        corrected_umis.append(final_umi[::3])
    return(corrected_umis)

test = correct_umi(barcodes)


In [524]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]


from collections import Counter
counts = []
n = 0
for umi in test:
    
    n +=1
    counts.append(levenshteinDistance("CTGACTGCTA", umi))

Counter(counts)


Counter({0: 165418,
         3: 35010,
         2: 92030,
         1: 41620,
         4: 20966,
         8: 2416,
         6: 6216,
         7: 5009,
         5: 10036,
         9: 176})

In [536]:

Counter(counts)[10]/n*100


0.0